In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

'''
Sourced from Tensorflow's tutorial on Variational Autoencoders.
'''

class CVAE(tf.keras.Model):
  """Convolutional variational autoencoder."""

  def __init__(self, latent_dim, shape_of_input, reg_coeff, dropout_rate):
    super(CVAE, self).__init__()
    #print(shape_of_input)
    self.latent_dim = latent_dim
    self.regularizer = tf.keras.regularizers.l2(l2=reg_coeff)
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=shape_of_input),
            tf.keras.layers.Conv2D(
                filters=32, kernel_size=(4,1), strides=1, activation='relu', kernel_regularizer=self.regularizer),
            tf.keras.layers.MaxPool2D(
                pool_size=(2, 2), strides=2, padding='valid'),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=(4,1), strides=1, activation='relu', kernel_regularizer=self.regularizer),
            tf.keras.layers.MaxPool2D(
                pool_size=(2, 1), strides=1, padding='valid'),
            tf.keras.layers.Flatten(),

            tf.keras.layers.Dense(124, kernel_regularizer=self.regularizer),

            tf.keras.layers.Dropout(dropout_rate),
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim, kernel_regularizer=self.regularizer),
        ]
    )

    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(units=shape_of_input[0]*shape_of_input[1], activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(shape_of_input[0], shape_of_input[1],1)),
            # tf.keras.layers.UpSampling2D(size=(2, 1)),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=(4,1), strides=1, padding='same',
                activation='relu',kernel_regularizer=self.regularizer),
            # tf.keras.layers.UpSampling2D(size=(2, 2)),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=(4,1), strides=1, padding='same',
                activation='relu', kernel_regularizer=self.regularizer),
            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=1, strides=1, padding='same'),
        ]
    )

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

In [52]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
#from TF_VAE_DNA import CVAE
import datetime as time
#from generate_dna_onehot import obtainDNAData
'''
Sourced from Tensorflow's tutorial on VAEs.
'''
optimizer = tf.keras.optimizers.Adam(1e-3)
def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)


def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)  #Try alternate loss functions, maybe interesting ones that penalize large mutations rather than small mutations
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)


#@tf.function
def train_step(model, x, optimizer):
  """Executes one training step and returns the loss.
  This function computes the loss and gradients, and uses the latter to
  update the model's parameters.
  """
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

## dataset_directory = "../../Datasets/Protein-DNA Complex/DNADataset.csv"
#dataset_directory = "../../Datasets/FinalDatasets/5SpeciesDNA.csv"
#dna_dataset_obtainer = obtainDNAData(dataset_directory)
#train_dataset, test_dataset = dna_dataset_obtainer.obtainData()
#train_dataset = tf.cast(tf.convert_to_tensor(train_dataset), dtype=tf.float32)
#test_dataset = tf.cast(tf.convert_to_tensor(test_dataset), dtype=tf.float32)

## train_dataset = []#TODO 100x4 for one hot encoding of DNA sequence, pad 0s to make sure have space for longest DNA sequence
#shape_of_input = tf.expand_dims(train_dataset[0], axis=2)
## shape_of_input = tf.expand_dims(shape_of_input, axis=3)

#shape_of_input = shape_of_input.shape

def train(epochs = 15, latent_dim=32, reg_coeff = 0.0001, dropout_rate = 0):
    model = CVAE(latent_dim, shape_of_input, reg_coeff, dropout_rate)
    for epoch in range(1, epochs + 1):
      start_time = time.datetime.now()
      _train_step = tf.function(train_step)
      for train_x in train_dataset:
        #shape_of_inputx = tf.expand_dims(train_x, axis=2)
        shape_of_inputx = tf.expand_dims(train_x, axis=0)
        _train_step(model, shape_of_inputx, optimizer)
      end_time = time.datetime.now()

      loss = tf.keras.metrics.Mean()
      for test_x in test_dataset:
        #shape_of_inputtest = tf.expand_dims(test_x, axis=2)
        shape_of_inputtest = tf.expand_dims(test_x, axis=0)
        loss(compute_loss(model, shape_of_inputtest))
      elbo = -loss.result()
      # display.clear_output(wait=False)
      print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
            .format(epoch, elbo, end_time - start_time))
      if elbo > -1800: 
            print('Elbo > -1800. Ending training.')
            break
    model.save_weights('Protein_VAE')
    return model

# hypers = {"latent_dim": [16, 32, 64, 128], "reg_coeff": [0.0001, 0.00001, 0.00005], "dropout_rate": [0.01, 0.001, 0.0001]}
# for l in range(3):
#     for r in range(3):
#         for d in range(3):
#             print("\n\nNew Epoch:\n\n")
#             latent = hypers["latent_dim"][l]
#             reg = hypers["reg_coeff"][r]
#             drop = hypers["dropout_rate"][d]
#             print('Latent Dimension: {}, Reg Coeff: {}, Dropout Rate: {}'
#                   .format(latent, reg, drop))

trained = train(latent_dim=40, reg_coeff=0.0001, dropout_rate=0.5)

# model.load_weights("dna_model")

Epoch: 1, Test set ELBO: -1825.4027099609375, time elapse for current epoch: 0:09:13.764576
Epoch: 2, Test set ELBO: -1814.2830810546875, time elapse for current epoch: 0:09:15.881803
Epoch: 3, Test set ELBO: -1804.86279296875, time elapse for current epoch: 0:09:13.043028
Epoch: 4, Test set ELBO: -1810.5401611328125, time elapse for current epoch: 1:02:17.860193
Epoch: 5, Test set ELBO: -1801.7003173828125, time elapse for current epoch: 0:10:33.320202
Epoch: 6, Test set ELBO: -1799.4583740234375, time elapse for current epoch: 0:10:19.771605
Elbo > 1800. Ending training.


In [31]:
import csv
from sklearn.preprocessing import OneHotEncoder

location = '/Users/AndrewHennes/Desktop/Code/Python/Class_Specific_Code/6.802/Project/Ensembl Datasets/New Datasets/5SpeciesProteins.csv'
data = np.array([row for row in csv.reader(open(location, 'r'))])[1:, 1:] ## Remove the x and y axis labels.
seq_len = len(data[0][1])//16*16
data = np.array([[aa for aa in row[1][:seq_len]] for row in data])
data = data[:len(data)//10]
data = data[:len(data)//50*50]

num_aa = 21
enc = OneHotEncoder(categories = [np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '_'], dtype='<U1') for i in range(seq_len)],
                    sparse = False, 
                    handle_unknown='ignore')


one_hot = enc.fit_transform(data).reshape(-1, seq_len, num_aa, 1).astype('float32')
train_dataset = one_hot[:len(one_hot)*4//5] # 80 Percent Training Dataset
test_dataset = one_hot[len(one_hot)*4//5:] # 20 Percent Testing Dataset

shape_of_input = train_dataset[0].shape
print(shape_of_input)

print("Training dataset shape is", train_dataset.shape)
print("Test dataset shape is", test_dataset.shape)

(496, 21, 1)
Training dataset shape is (7520, 496, 21, 1)
Test dataset shape is (1880, 496, 21, 1)


In [43]:
for ind1, training_rate in enumerate([1e-3, 1e-4]):
  for ind2, dropout_rate in enumerate([0.0, 0.5, 0.9]):
    for ind3, latent_dim in enumerate([40, 80]):
      print("For a training_rate of %s, a dropout rate of %s, and a latent dim of %s, the loss over 10 epochs is:" % (training_rate, dropout_rate, latent_dim))
      model = train(epochs = 10, latent_dim = latent_dim, reg_coeff = 0.0001, dropout_rate = dropout_rate)
      # results[(training_rate, dropout_rate, latent_dim)] = losses

For a training_rate of 0.001, a dropout rate of 0.0, and a latent dim of 40, the loss over 10 epochs is:
Epoch: 1, Test set ELBO: -1927.2818603515625, time elapse for current epoch: 0:09:45.108447
Epoch: 2, Test set ELBO: -1906.0029296875, time elapse for current epoch: 0:09:37.341483
Epoch: 3, Test set ELBO: -1904.478271484375, time elapse for current epoch: 0:09:42.298420
Epoch: 4, Test set ELBO: -1903.0545654296875, time elapse for current epoch: 0:09:33.659189
Epoch: 5, Test set ELBO: -1902.5595703125, time elapse for current epoch: 0:09:31.650460
Epoch: 6, Test set ELBO: -1901.9940185546875, time elapse for current epoch: 0:09:33.971579
Epoch: 7, Test set ELBO: -1901.787109375, time elapse for current epoch: 0:09:33.180841
Epoch: 8, Test set ELBO: -1901.412109375, time elapse for current epoch: 0:09:31.668406
Epoch: 9, Test set ELBO: -1901.3865966796875, time elapse for current epoch: 0:09:26.313177
Epoch: 10, Test set ELBO: -1901.3507080078125, time elapse for current epoch: 0:09

Epoch: 1, Test set ELBO: -1927.266845703125, time elapse for current epoch: 0:09:33.533038
Epoch: 2, Test set ELBO: -1913.327392578125, time elapse for current epoch: 0:09:27.483162
Epoch: 3, Test set ELBO: -1909.966796875, time elapse for current epoch: 0:09:24.922501
Epoch: 4, Test set ELBO: -1908.4083251953125, time elapse for current epoch: 0:09:29.020498
Epoch: 5, Test set ELBO: -1909.1766357421875, time elapse for current epoch: 0:09:19.965492
Epoch: 6, Test set ELBO: -1909.5797119140625, time elapse for current epoch: 0:09:18.721761
Epoch: 7, Test set ELBO: -1910.177734375, time elapse for current epoch: 0:09:20.467228
Epoch: 8, Test set ELBO: -1910.322265625, time elapse for current epoch: 0:09:18.482076
Epoch: 9, Test set ELBO: -1911.418212890625, time elapse for current epoch: 0:09:15.542400
Epoch: 10, Test set ELBO: -1912.019287109375, time elapse for current epoch: 0:09:31.342118
For a training_rate of 0.0001, a dropout rate of 0.5, and a latent dim of 80, the loss over 10 

In [56]:
aas = ['*', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
sample = model.sample().numpy()[0]
sample = np.argmax(sample, axis = 1)
print(list(sample.flatten()))

[8, 0, 8, 8, 8, 12, 12, 12, 12, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 5, 5, 5, 5, 15, 15, 15, 15, 15, 9, 15, 9, 3, 3, 14, 0, 15, 0, 0, 0, 0, 14, 14, 14, 14, 5, 5, 5, 5, 5, 5, 17, 17, 17, 17, 0, 0, 0, 0, 9, 9, 12, 12, 0, 0, 0, 16, 16, 16, 5, 2, 2, 2, 15, 15, 15, 9, 9, 11, 9, 11, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 0, 0, 2, 17, 17, 17, 13, 13, 13, 13, 13, 3, 8, 4, 3, 16, 17, 0, 0, 0, 0, 0, 0, 0, 3, 9, 9, 9, 9, 9, 9, 14, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 16, 16, 9, 9, 15, 15, 15, 15, 0, 0, 0, 3, 3, 3, 3, 3, 15, 3, 3, 9, 9, 9, 14, 12, 17, 7, 7, 7, 7, 7, 16, 3, 16, 3, 3, 12, 8, 0, 0, 0, 0, 0, 0, 0, 7, 7, 2, 8, 8, 8, 15, 8, 9, 9, 12, 12, 12, 2, 9, 9, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 15, 16, 15, 15, 15, 3, 3, 3, 3, 3, 17, 12, 17, 17, 17, 15, 15, 5, 16, 5, 16, 16, 9, 9, 9, 9, 7, 7, 8, 8, 8, 12, 12, 12, 15, 12, 12, 15, 15, 15, 15, 15, 7, 7, 2, 2, 3, 3, 5, 5, 5, 5, 5, 0, 0, 0, 0, 9, 9, 9, 9, 9, 17, 15, 9, 14, 9, 14, 16, 16, 8, 15, 15, 15, 15, 15, 15, 15, 5, 5, 5, 5, 12, 12, 12, 9, 7, 9, 11, 11, 11, 11, 11, 8, 0